In [1]:
import pandas as pd
import re

In [2]:
cta_train_gt = pd.read_json('train.table_col_type.json')
cta_test_gt = pd.read_json('test.table_col_type.json')

In [5]:
len(cta_test_gt)

4764

In [6]:
len(cta_train_gt)

397098

In [195]:
cta_test_gt

,0,1,2,3,4,5,6,7
0,27295818-29,2010–11 rangers f.c. season,27295818,overall,,"[competition, started round, final position / ...","[[[[0, 0], [26980923, 'Scottish Premier League...","[[time.event], [time.event], [time.event]]"
1,27298128-1,kagawa at-large district (house of councillors),27298128,elected councillors,,[election year],"[[[[0, 0], [31629558, '1947']], [[1, 0], [3162...",[[time.event]]
2,27298635-1,list of museums in northumberland,27298635,,see also,[town/city],"[[[[0, 0], [129195, 'Alnwick']], [[1, 0], [208...",[[location.location]]
3,27308318-1,list of museums in north yorkshire,27308318,,defunct museums,"[town/city, region]","[[[[0, 0], [2400278, 'Aldborough']], [[1, 0], ...","[[location.location], [location.location]]"
4,6675886-1,indy japan 300,6675886,past winners,usac championship car history (non-championsh...,"[season season, driver driver, team team, chas...","[[[[2, 0], [5710555, '1998']], [[3, 0], [64984...","[[sports.sports_league_season, time.event], [p..."
...,...,...,...,...,...,...,...,...
4759,8086435-1,112 discography,8086435,studio albums,"list of studio albums, with selected chart pos...",[title title],"[[[[0, 0], [2226709, '112']], [[1, 0], [533977...",[[music.album]]
4760,808980-1,list of rulers of austria,808980,house of babenberg,,"[name, marriages, death]","[[[[0, 0], [1100652, 'Leopold I the Illustriou...","[[people.person, people.family_member, royalty..."
4761,808980-2,list of rulers of austria,808980,house of babenberg,,"[name, marriages, death]","[[[[0, 0], [1116423, 'Henry II Jasomirgott (He...","[[people.person, people.family_member, royalty..."
4762,809525-4,nba conference finals,809525,division finals,,"[year, champion, runner-up]","[[[[0, 0], [8569886, '1949']], [[1, 0], [86170...","[[award.competition, time.event], [basketball...."


In [ ]:
all_labels = ['soccer.football_league', 'government.government_office_or_title', 'organization.non_profit_organization', 'olympics.olympic_games', 'cvg.cvg_genre', 'ice_hockey.hockey_position', 'tv.tv_network', 'aviation.airline', 'american_football.football_conference', 'soccer.football_world_cup', 'american_football.football_coach', 'military.military_unit', 'military.military_post', 'music.media_format', 'tv.tv_personality', 'baseball.baseball_team', 'cvg.cvg_developer', 'soccer.football_award', 'ice_hockey.hockey_team', 'tv.tv_writer', 'meteorology.tropical_cyclone_season', 'soccer.fifa', 'cvg.cvg_publisher', 'baseball.baseball_player', 'sports.sports_championship', 'soccer.football_team_manager', 'sports.golfer', 'baseball.baseball_position', 'military.rank', 'cvg.cvg_platform', 'music.musical_group', 'amusement_parks.ride', 'music.genre', 'music.lyricist', 'music.record_label', 'meteorology.tropical_cyclone', 'aviation.airport']

In [ ]:
len(all_labels)

37

In [ ]:
""" df =  pd.read_json('test.table_col_type.json')
df = df[list(cta_test_gt.keys())]
print(df) """

In [196]:
test_examples = []

for primary_row_index, primary_row_values in cta_test_gt.iterrows():

    table_data = []
    sampled_labels = []

    # print(primary_row_values[0])

    all_columns = []

    for col_index, col_labels in enumerate(primary_row_values[7]):

        sampled_labels_column = []
        for label in col_labels:
            if label in all_labels:
                sampled_labels_column.append(label)
        
        if len(sampled_labels_column) != 0:
            sampled_labels.append(sampled_labels_column)
            row_col = []

            for row_index in primary_row_values[6][col_index][:5]:
                # print(f"row_index: {row_index}")
                row_col.append(row_index[1][1])
            if len(primary_row_values[6][col_index]) < 5:
                for l in range(5-len(primary_row_values[6][col_index])):
                    row_col.append(" ")
            all_columns.append(row_col)

    col_labels = " || ".join(f"Column {i+1}" for i in range(0, len(all_columns))) 
    table_data.append(col_labels)

    # print(all_columns)

    if len(all_columns) > 1:
        for i in range(len(all_columns[0])):
            row_string = []
            for j in range(len(all_columns)):
                row_string.append(all_columns[j][i])
            table_data.append(" || ".join(row_string).strip())
            # print(row_string)
    elif len(all_columns) == 1:
        row_string = []
        for j in range(len(all_columns[0])):
            row_string.append(all_columns[0][j])
        table_data.append(" ||\n".join(row_string))

    # print(table_data)
    
    if sampled_labels:
        if len(all_columns) > 1:
            formatted_output = " ||\n".join(table_data)
            formatted_output += " ||"
        else:
            #formatted_output = table_data
            formatted_output = " ||\n".join(table_data)
           
        test_examples.append([primary_row_values[0], formatted_output, sampled_labels])



In [ ]:
print(test_examples[3][1])

In [197]:
test_examples

[['33879379-1',
  'Column 1 ||\nPremier League ||\nPremier League ||\nLeague One ||\nChampionship ||\n ',
  [['soccer.football_league']]],
 ['15664872-1',
  'Column 1 || Column 2 ||\nUni-President Lions || Wei Chuan Dragons ||\nWei Chuan Dragons || Sinon Bulls ||\nUni-President Lions || Sinon Bulls ||\nBrother Elephants || Uni-President Lions ||\nSinon Bulls || Uni-President Lions ||',
  [['baseball.baseball_team'], ['baseball.baseball_team']]],
 ['56602-2',
  'Column 1 ||\nFootball ||\nFootball ||\nFootball ||\nIce hockey ||\nIce hockey',
  [['cvg.cvg_genre']]],
 ['29006961-1',
  'Column 1 || Column 2 ||\nJoel Skinner || Managed AA- Akron Aeros ( CLE ) ||\nRick Rodriguez || Pitching coach for Sacramento ||\nGerald Perry || Hitting coach for AAA- Pawtucket Red Sox ( BOS ) ||\n|| ||\n|| ||',
  [['baseball.baseball_player'], ['baseball.baseball_team']]],
 ['29020787-10',
  'Column 1 ||\nColumbus Blue Jackets ||\nNew York Rangers ||\nBuffalo Sabres ||\nCalgary Flames ||\nNew Jersey Devils

In [198]:
len(test_examples)

622

In [199]:
train_examples = []

for primary_row_index, primary_row_values in cta_train_gt.iterrows():

    table_data = []
    sampled_labels = []

    # print(primary_row_values[0])

    all_columns = []

    for col_index, col_labels in enumerate(primary_row_values[7]):

        sampled_labels_column = []
        for label in col_labels:
            if label in all_labels:
                sampled_labels_column.append(label)
        
        if len(sampled_labels_column) != 0:
            sampled_labels.append(sampled_labels_column)
            row_col = []

            for row_index in primary_row_values[6][col_index][:5]:
                # print(f"row_index: {row_index}")
                row_col.append(row_index[1][1])
            if len(primary_row_values[6][col_index]) < 5:
                for l in range(5-len(primary_row_values[6][col_index])):
                    row_col.append(" ")
            all_columns.append(row_col)

    col_labels = " || ".join(f"Column {i+1}" for i in range(0, len(all_columns))) 
    table_data.append(col_labels)

    # print(all_columns)

    if len(all_columns) > 1:
        for i in range(len(all_columns[0])):
            row_string = []
            for j in range(len(all_columns)):
                row_string.append(all_columns[j][i])
            table_data.append(" || ".join(row_string).strip())
            # print(row_string)
    elif len(all_columns) == 1:
        row_string = []
        for j in range(len(all_columns[0])):
            row_string.append(all_columns[0][j])
        table_data.append(" ||\n".join(row_string))

    # print(table_data)
    
    if sampled_labels:
        if len(all_columns) > 1:
            formatted_output = " ||\n".join(table_data)
            formatted_output += " ||"
        else:
            #formatted_output = table_data
            formatted_output = " ||\n".join(table_data)
           
        train_examples.append([primary_row_values[0], formatted_output, sampled_labels])


In [200]:
train_examples

[['27284790-1',
  'Column 1 ||\nPersian Gulf Cup ||\nPersian Gulf Cup ||\nAzadegan League ||\nPersian Gulf Cup ||\nPersian Gulf Cup',
  [['soccer.football_league']]],
 ['27285187-1',
  'Column 1 ||\nPro League ||\nPro League ||\nPro League ||\nPro League ||\nDivision 1',
  [['soccer.football_league']]],
 ['27287912-2',
  'Column 1 ||\nRon Villone ||\nBob Wolcott ||\nChris Widger ||\nAndy Sheets ||\nTim Davis',
  [['baseball.baseball_player']]],
 ['27289781-1',
  "Column 1 ||\nLigat ha'Al ||\nLigat ha'Al ||\nLigat ha'Al ||\nLiga Leumit ||\nLigat ha'Al",
  [['soccer.football_league']]],
 ['2729003-2',
  'Column 1 ||\nNine Network ||\nARD ||\nRTL plus ||\nRCTI ||\nRCTI',
  [['tv.tv_network']]],
 ['2729393-1',
  'Column 1 || Column 2 ||\nBingo Airways || Seasonal Charter: Antalya , Hurghada , Sharm el Sheikh , Enfidha ||\nEnter Air || Seasonal Charter: Antalya , Burgas , Chania , Dubrovnik , Heraklion , Rhodes , Lanzarote , Tenerife-South ||\nFreebird Airlines || Seasonal Charter: Antalya 

In [201]:
len(train_examples)

32557

In [202]:
import pickle

In [203]:
file_name='../WikiData CTA/wiki-cta-test-table.pkl'
f = open(file_name,'wb')
pickle.dump(test_examples,f)
f.close()

In [204]:
file_name='../WikiData CTA/wiki-cta-train-table.pkl'
f = open(file_name,'wb')
pickle.dump(train_examples,f)
f.close()